In [1]:
! pip list | grep pyspark
! pip install pyspark
from pyspark import *
from pyspark import SQLContext
import urllib
from pyspark.sql.types import *

pyspark                  2.4.4      


In [2]:
sc = SparkContext(master="local", appName="test")
sc

<SparkContext master=local appName=test>

In [3]:
data = urllib.request.urlretrieve('http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz', 'kddcup.data_10_percent.gz')
data_file = './kddcup.data_10_percent.gz'

df = sc.textFile(data_file)
print("data_loaded")

data_loaded


In [4]:
sqlContext = SQLContext(sc)
sqlContext

In [0]:
df = df.map( lambda x: x.split(','))

In [6]:
structure = StructType([
        StructField("duration", IntegerType(), True),
        StructField("protocol_type", StringType(), True),
        StructField("service", StringType(), True),
        StructField("flag", StringType(), True),
        StructField("src_bytes",IntegerType(), True),
        StructField("dst_bytes",IntegerType(), True),
        StructField("interactions", StringType(), True),
])
structure

StructType(List(StructField(duration,IntegerType,true),StructField(protocol_type,StringType,true),StructField(service,StringType,true),StructField(flag,StringType,true),StructField(src_bytes,IntegerType,true),StructField(dst_bytes,IntegerType,true),StructField(interactions,StringType,true)))

In [0]:
formatedData = df.map(lambda key: (int(key[0]), key[1], key[2], key[3], int(key[4]), int(key[5]), key[-1]))

In [0]:
data = sqlContext.createDataFrame(formatedData, structure)

In [9]:
data.show(10)

+--------+-------------+-------+----+---------+---------+------------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|interactions|
+--------+-------------+-------+----+---------+---------+------------+
|       0|          tcp|   http|  SF|      181|     5450|     normal.|
|       0|          tcp|   http|  SF|      239|      486|     normal.|
|       0|          tcp|   http|  SF|      235|     1337|     normal.|
|       0|          tcp|   http|  SF|      219|     1337|     normal.|
|       0|          tcp|   http|  SF|      217|     2032|     normal.|
|       0|          tcp|   http|  SF|      217|     2032|     normal.|
|       0|          tcp|   http|  SF|      212|     1940|     normal.|
|       0|          tcp|   http|  SF|      159|     4087|     normal.|
|       0|          tcp|   http|  SF|      210|      151|     normal.|
|       0|          tcp|   http|  SF|      212|      786|     normal.|
+--------+-------------+-------+----+---------+---------+------------+
only s

In [0]:
data.createOrReplaceTempView("interactions")

In [12]:
le139 = data[(data['duration']>1000) & (data['dst_bytes'] == 0) & (data['protocol_type'] == "tcp")]
le139.count()

139


In [14]:
import time
protocol = data[['protocol_type']].groupby(['protocol_type']).count()
start_time= time.time()
protocol.show()
print("Temps d'execution : %s secondes ---" % (time.time() - start_time))

+-------------+------+
|protocol_type| count|
+-------------+------+
|          tcp|190065|
|          udp| 20354|
|         icmp|283602|
+-------------+------+

Temps d'execution : 9.32252287864685 secondes ---


In [16]:
durabyte = data[(data['duration']>1000) & (data['dst_bytes'] == 0)].groupby(['protocol_type']).count()
start_time= time.time()
durabyte.show()
print("Temps d'execution : %s secondes ---" % (time.time() - start_time))

+-------------+-----+
|protocol_type|count|
+-------------+-----+
|          tcp|  139|
+-------------+-----+

Temps d'execution : 8.759639978408813 secondes ---


In [17]:
data.rdd.map(lambda x : "Duration: {0}, Dest. bytes: {1}".format(x.dst_bytes,x.duration)).take(5)

['Duration: 5450, Dest. bytes: 0',
 'Duration: 486, Dest. bytes: 0',
 'Duration: 1337, Dest. bytes: 0',
 'Duration: 1337, Dest. bytes: 0',
 'Duration: 2032, Dest. bytes: 0']

In [0]:
from pyspark.sql.functions import udf

def attack_or_normal_func(s):
    return "normal" if s == "normal." else "attack"

In [28]:
label = udf(attack_or_normal_func, StringType())
data = data.withColumn('label', label(data['interactions']))
start_time= time.time()
data.select('label').groupby('label').count().show()
print("Temps d'execution : %s secondes ---" % (time.time() - start_time))

Py4JJavaError: ignored

In [34]:
data.printSchema()

root
 |-- duration: integer (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- interactions: string (nullable = true)
 |-- label: string (nullable = true)



In [42]:
start_time= time.time()
data.select('label','protocol_type').groupby('label','protocol_type').count().show()
print("Temps d execution : %s secondes ---" % (time.time() - start_time))

Py4JJavaError: ignored